### Imports

In [13]:
%run gpt_power_point_creator_mul_slides.ipynb
%run summarization_training.ipynb

import openai
import wikipediaapi
from google_images_download import google_images_download
import os
import shutil

### Set lenght of bulletpoints and max. number of slides

In [14]:
max_len = 150  
max_slide_num = 5

### Set wikipedia language

In [15]:
wiki_wiki = wikipediaapi.Wikipedia(language='en', extract_format=wikipediaapi.ExtractFormat.WIKI)

### Get topic

In [16]:
while True:
    # Inputs
    prompt = input("Topic: ")

    # Define the wikipedia page
    p_wiki = wiki_wiki.page(prompt)

    # dictionary of the wikipedia page
    def create_dict(page):
        dict = {}
        dict[page.title] = page.summary
        for s in page.sections:
            dict[s.title] = s.text

            try:
                if s.text == "":
                    dict[s.title] = s.sections[0].text
            except:
                pass
        return dict

    wiki_dict = create_dict(p_wiki)
    try:
        wiki_dict.pop("See also")
    except:
        pass
    try:
        wiki_dict.pop("References")
    except:
        pass
    try:
        wiki_dict.pop("External links")
    except:
        pass
    try:
        wiki_dict.pop("Further reading")
    except:
        pass
    try:
        wiki_dict.pop("Notes")
    except:
        pass

    new_wiki_dict = {}
    slide_num = 0
    for key in wiki_dict:
        slide_num += 1
        if slide_num <= max_slide_num:
            new_wiki_dict[key] = wiki_dict[key]

    # Print Subtopics
    print("\nFound Subtopics: ")
    for key in new_wiki_dict:
        print(key)
    print("\n")

    verify = input("Do you want a power-point for these topics? (y/n): ")
    if verify.lower() == "y":
        break


Topic: ferrari

Found Subtopics: 
Ferrari
History
Motorsport
Road cars
Identity


Do you want a power-point for these topics? (y/n): y


### Openai key

In [17]:
with open("openai_key.txt") as file:
    key = file.read()
    openai.api_key = key

gpt_sum = summarization_training.create_sum_model()

dict_for_pptx = {}

for key in new_wiki_dict:
    # Get GPT output
    output = gpt_sum.submit_request(new_wiki_dict[key])
    output = output.choices[0].text[8:]

    # Crop if too long
    if len(output) > max_len:
        output = output[:max_len]
        to_cut = ""
        for i in reversed(range(1, len(output))):
            if output[i] == ".":
                output = output.replace(to_cut, "")
                break
            to_cut = output[i] + to_cut

    # Print Points
    print("\nSummarized points for " + key + ":")
    for sentence in output.split(". "):
        print("    - " + sentence)

    dict_for_pptx[key] = output

# Delete download folder if it exists
try:
    shutil.rmtree('downloads')
except FileNotFoundError:
    pass


Summarized points for Ferrari:
    - Ferrari S.p.A
    - is an Italian luxury sports car manufacturer.

Summarized points for History:
    -  Enzo Ferrari was not initially interested in the idea of producing road cars when he formeduderia Ferrari in 1929, with headquarters in Modena.

Summarized points for Motorsport:
    - Since its beginnings, Ferrari has been involved in motorsport.

Summarized points for Road cars:
    -  The first vehicle made with the Ferrari name was the 125 S
    - Only two of this small two-seat sports/racing V12 car were made.

Summarized points for Identity:
    -  The Cavallino Rampante is the symbol of Ferrari
    - Cavallino Magazine uses the name, but not the logo.


### Download an image for the keyword

In [18]:
response = google_images_download.googleimagesdownload()
arguments = {"keywords": prompt, "limit": max_slide_num+2, "print_urls": True, format: "jpg"}
response.download(arguments)


img_list = os.listdir("downloads/" + prompt)
img_dict = {}

counter = 0
try:
    for key in dict_for_pptx:
        img_dict[key] = "downloads/" + prompt + "//" + img_list[counter]
        counter += 1
except:
    pass


Item no.: 1 --> Item name = ferrari
Evaluating...
Starting Download...
Image URL: https://carconfigurator.ferrari.com/assets/cars/portofinom/packages/default/car-ferrari-portofino-m_splash.jpg
Completed Image ====> 1.car-ferrari-portofino-m_splash.jpg
Image URL: https://imgr1.auto-motor-und-sport.de/Ferrari-F8-Tributo-bigMobileWide-1054d77e-1432047.jpg
Completed Image ====> 2.ferrari-f8-tributo-bigmobilewide-1054d77e-1432047.jpg
Image URL: https://upload.wikimedia.org/wikipedia/commons/1/13/Red_2019_Ferrari_SF90_Stradale_%2848264238897%29_%28cropped%29.jpg
Completed Image ====> 3.red_2019_ferrari_sf90_stradale_%2848264238897%29_%28cropped%29.jpg
Image URL: https://m.faz.net/media1/ppmedia/aktuell/technik-motor/2932592337/1.6841143/mmobject-still_full/der-neue-ferrari-sf90-stradale.jpg
Completed Image ====> 4.der-neue-ferrari-sf90-stradale.jpg
Image URL: https://lookaside.fbsbx.com/lookaside/crawler/media/?media_id=10159405266101970
Invalid image format 'text/html'. Skipping...
Image U

### Create PowerPoint with multiple slides

In [19]:
gpt_power_point_creator_mul_slides.create_power_point_slides_from_gpt(dict_for_pptx, img_dict, prompt)
